## Story so far...

#### 1. About the Class

* We have a class (Triangle) 
    * It has no property or method
    
* The role of class is to 
    1. create the object :   **t=Triangle()**
    2. identify object type:  **is_instance(t,Triangle)**
    
#### 2. About the Object

* we can add new properties to the object after it is created
    * it can be considered as a group of properties associated with single entity

#### 3. About behaviors

* we can defines functions that are supposed to operate on a given object type
* we can validate object type and required properties before taking actions


#### Problem

* different objects (e.g. Cricle, Triangle, Rectangle) have similar behaviors (eg. area(), perimeter() )
* python doesn't support overloading
    * it overwrites
* we need a way to represent same behavior for different type of objects

#### Solutions

1. use name prefixes to change behavior names
    * circle_perimeter()
    * triangle_perimeter()

2. Use Modules to represent the behavior



## Solution 3  Use Class as Replacement of Module

### 3.1 Even class is an Object

In [4]:
class Circle:
    pass

print(type(Circle))

<class 'type'>


#### 3.2 Methods are also objects (We already know this)

In [2]:
import math
NAN = float('nan')

def perimeter(circle):
    return 2*math.pi*circle.radius if circle.radius>0 else NAN

def create(radius):
    c=Circle()
    c.radius=radius
    return c

In [3]:
print(type(perimeter))
print(type(create))

<class 'function'>
<class 'function'>


#### 3.3 an object can have other object as property

* circle object can have side
* **Circle class object** can have perimeter

In [6]:
Circle.perimeter=perimeter
Circle.create=create

### I can do the same for Triangle
* The next set of create() and perimeter() will overrwrite previous global periemter() and create() that we created for circle
* but it will NOT overwrite the methods attached to Circle classes
    * it will only overwrite global names

In [10]:
import math
NAN = float('nan')


class Circle:
    pass


def perimeter(circle):
    return 2*math.pi*circle.radius if circle.radius>0 else NAN

def create(radius):
    c=Circle()
    c.radius=radius
    return c



Circle.perimeter=perimeter
Circle.create=create

class Triangle:
    pass

# this create will overwrite previous create
# but it will not effect Circle.create
def create(s1,s2,s3):
    t=Triangle()
    t.s1=s1
    t.s2=s2
    t.s3=s3
    return t

def perimeter(t):
    return t.s1+t.s2+t.s3

Triangle.create=create # new create
Triangle.perimeter=perimeter # new perimeter


In [11]:
t=Triangle.create(3,4,5)
Triangle.perimeter(t)

12

In [12]:
c=Circle.create(7)
Circle.perimeter(c)

43.982297150257104

### What have we achieved 

1. Earlier a class was used only for 
    a. creating the object
    b. providing object a type identification

2. Now we are using class to group methods related to the object that class
    * Here it is acting more like a Module
        * A module is also an object that groups all methods present in python script



## Simplified class syntax

* instead of creating a global function and then assigning to the class, we can also add those methods in the body of the class definition
* This is a semantic convinience which has exact same menaing as the code above.


In [17]:
class Circle:
    def create(radius):
        c=Circle()
        c.radius=radius
        return c
    
    def is_valid(circle):
        return isinstance(circle, Circle) and circle.radius>0
    
    def perimeter(circle):
        return 2*math.pi*circle.radius if Circle.is_valid(circle) else NAN
    
    def area(circle):
        return math.pi*circle.radius*circle.radius if Circle.is_valid(circle) else NAN
    
    def info(circle):
        return f"Circle({circle.radius})" if Circle.is_valid(circle) else "Circle(invalid)"
    
    def draw(circle,surface='Screen'):
    
        print(f'drawing {Circle.info(circle)} on {surface}')

In [18]:
def test_circle(radius):
    c=Circle.create(radius)
    Circle.draw(c,'Paper')
    if Circle.is_valid(c):
        print('Perimeter', Circle.perimeter(c))
        print('Area', Circle.area(c))

    print()

In [19]:
test_circle(7)

test_circle(-7)

drawing Circle(7) on Paper
Perimeter 43.982297150257104
Area 153.93804002589985

drawing Circle(invalid) on Paper



## Magic Python Syntax for object methods

* Generally a class contains methods that operate of the object of that class
    * they represent object's behavior
    * Example
        * Circle.area calculates area of the circle object 
        * Cricle.draw draws the circle object 

#### But the symantic of method call is redundant

* consider the code snippet below

```python
c = Circle.create(7)

p = Circle.perimeter(c)

Circle.draw(c, "canvas" )

```

* How many times '**circle**' appears in call of **perimeter()** function?
    1. Circle (the class)
    2. c is the Circle object

* This is redundant code.

* Python tends to simplify this code.

```python
p = c.perimeter()  # Circle.perimeter(c)
```

* This semantic works only if
    * a class method takes object of the same class as the first parameter

In [20]:
c=Circle.create(7)

p1= Circle.perimeter(c)  # same as below

p2 = c.perimeter() # internally converted to Circle.perimeter(c)

print(p1,p2)

43.982297150257104 43.982297150257104


#### This will also work with Triangle created with external function

In [21]:
t=Triangle.create(3,4,5)

t.perimeter() # Triangle.perimeter(t)

12

#### What if the function takes more than one parameter?

```python
c=Circle.create(7)

Circle.draw(c,"paper")
```

* we can still use the object notation
    * we will pass one parameter less.

* the object invoking the function is passed as the first argument to that function

* any argument we pass is shifted by one place



In [22]:
c=Circle.create(7)

c.draw('Canva') # Circle.draw(c,'Cavas')

drawing Circle(7) on Canva


### This syntax doesn't work if class method doesn't take class object as first parameter

* We can't use this syntax with create function below


```python
c = Circle.create(7)
```


* create function takes a number (int/float). it doesn't take circle
* we can't use this object notation with current version of create

In [23]:
c = Circle()

c.create(7) # Circle.create(c,7)

TypeError: create() takes 1 positional argument but 2 were given

### A word on convention

* it is tiresome and redundant to say 'class method that takes class object as first parameter'
* python community created a convention to avoid this long phrase
* we name this first parameter as **self**
    * It can be compared with **this** of c++ styles languages
        * But it is **NOT** exactly like **this** 


#### self vs this

* 'this' is a keyword, 'self' is a user defined convention
* 'this' is always called 'this', you can use any name inplace of 'self'
    * in our example we have used 'circle'

* 'this' is passed by compiler implcitly, 'self' is always passed explicitly

* writing **this.raidus** is mostly optional (needed only in case of ambiguity)
    * **self.radius** is required everytime. It is NOT optional



## How do I make the below code work?



In [24]:
c=Circle()

c.create(7)

c.draw('paper')

TypeError: create() takes 1 positional argument but 2 were given

### Redefining Circle class

In [40]:
class Circle:
    def create(self,radius):
        #c=Circle()
        self.radius=radius
        #return c
    
    def is_valid(self):
        return isinstance(self, Circle) and self.radius>0
    
    def perimeter(self):
        return 2 * math.pi * self.radius if self.is_valid() else NAN
    
    def area(self):
        return math.pi * self.radius * self.radius if Circle.is_valid(self) else NAN
    
    def info(self):
        return f"Circle({self.radius})" if self.is_valid() else "Circle(invalid)"
    
    def draw(self,surface='Screen'):
        if self.is_valid():
            print(f'drawing {self.info()} on {surface}')
        else:
            print(f"can't draw invalid circle")

In [41]:
def test_circle(radius):
    #c = Circle.create(radius)

    c=Circle()
    c.create(radius)
    c.draw('Paper')
    if c.is_valid():
        print('Perimeter', c.perimeter())
        print('Area', c.area())

    print()

In [42]:
test_circle(7)

test_circle(-1)

drawing Circle(7) on Paper
Perimeter 43.982297150257104
Area 153.93804002589985

can't draw invalid circle



### When is the circle Created?



In [43]:
c = Circle()

c.create(7)

c.perimeter()

43.982297150257104

#### When is the circle created -- on line #1 or line #3?

##### if it is created on line #1

* what is the raidus of this circle?
* what will be the perimeter on line #2?
* how will the circle look like on line #2?
* **how can we have a circle without radius?**
* what are we doning on line #3 when we say **c.create()** 

#### if it is created on line #3

* what have we done on line #1?
* what will be the **type(c)** on line #2
* what will be the **id(c)** one line#2
* if object doesn't exist, how is it being called a **circle** or has an **id**?


### What is the main problem?

* There are two different circles we are talking about

#### 1. Geometric circle

* doesn't exist till it has a radius (line #3)
* programming is all about representing this **domain object**

#### 2. Python circle (in memory/ semantic)

* gets created using **constructor syntax** 
* it is python's syntax. but domain object doesn't exist anymore

### Ideally there should be no gap between the two stages.


## Python's internal process for creating an object

* when we try to create a python object 

```python
c = Circle()
```
* We are calling the **constructor**  called **Circle** 
* This constructor has same name as that of class 
    * It is same as C++ style languages


* This constructor internally calls two functions
* It is conceptually similar to below code

```python
class Circle:
    def Circle():
        c=Circle.__new__() # 1. allocates memory. This is python's cricle
        c.__init__() # 2. initialized the values. This is geometric circle requirement
        return c
```

* In python we never override default constructor
* We rarely need to make changes to **\_\_new\_\_()**
* But we can define our own **\_\_init\_\_(self)**


In [44]:
class Circle:
    def __init__(self):
        print('circle.__init__(self) called')

In [45]:
c=Circle() # internally calls __init__

circle.__init__(self) called


### we can pass parameters to \_\_init\_\_

* default \_\_init\_\_ takes only 1 parameter (self)
* but we can overwrite \_\_init\_\_ to take more parameters
* since we don't call \_\_init\_\_ ourself, we need to pass it to constructor


In [46]:
class Circle:
    def __init__(self, radius):
        self.radius=radius

### Now we can't call Circle constructor without parameter

In [47]:
c=Circle() # internall calls c.__init__()

TypeError: __init__() missing 1 required positional argument: 'radius'

In [48]:
c=Circle(7) # internally calls c.__init__(7) --> Circle.__init__(c,7)

print(c.radius)

7


### \_\_init\_\_ vs constructor

* **\_\_init\_\_** is often referred as **constructor**
    * it has a similar job as a c++ style constructor
        * initalize the object

* But \_\_init\_\_ is not a constructor.
    * it is a special method called from within the constructor
    * it has subtle differences from the constructor  of C++ style languages


## VERY IMPORTANT

* A Python class NEVER defines an object's properties (fields/state/values).
    * they are added as convinience by \_\_init\_\_ method
        * \_\_init\_\_ is more a convinience that semantic compulsion

* we don't declare them as part of class definition
* we can still add additional properties after the object is created

In [50]:
c=Circle(7)
c.color='blue'

[m for m in dir(c) if not m.startswith('__')]

['color', 'radius']

In [51]:
c.perimeter()

AttributeError: 'Circle' object has no attribute 'perimeter'

#### Complete circle code

In [64]:
class Circle:
    def __init__(self,radius):
        self.radius=radius
        
        
    
    def is_valid(self):
        return isinstance(self, Circle) and self.radius>0
    
    def perimeter(self):
        return 2 * math.pi * self.radius if self.is_valid() else NAN
    
    def area(self):
        return math.pi * self.radius * self.radius if Circle.is_valid(self) else NAN
    
    def info(self):
        return f"Circle({self.radius})" if self.is_valid() else "Circle(invalid)"
    
    def draw(self,surface='Screen'):
        if self.is_valid():
            print(f'drawing {self.info()} on {surface}')
        else:
            print(f"can't draw invalid circle")

In [65]:
c=Circle(7)
c.draw('paper')

drawing Circle(7) on paper
